# Exploratory Testing Notebook for Asset Time Machine App

In [1]:
# base packages
import numpy as np
import pandas as pd
import datetime
#from janitor import clean_names, remove_empty

## Testing Data Connections

### Pulling from Yahoo Finance's API

In [2]:
import fix_yahoo_finance as yf

# setting a couple variables that will be parameters in the app (probably):

# getting today's date so we pull stock data relative to today
date_today = datetime.datetime.today()
# formatting today's date appropriately for yf.download() call
formatted_date_today = date_today.strftime('%Y-%m-%d')
# setting name of asset to pull (will be callback result in app)
present_asset_name = 'AAPL'
# setting start date of period for which to pull data...
# gunna keep this relatively recent until zach fixes the aggregation functions otherwise it takes forever
start_date = '2018-01-01'

initial_data_pull_df = (
    # pulling historical data for Apple using fixed yahoo finance API
    yf.download(
        tickers=present_asset_name,
        start=start_date,
        end=formatted_date_today,
        progress=False)
    # using janitor to clean the column names
    # janitor won't install, so for now commented out and will manually fix names
    #.pipe(clean_names)
)

#new_col_names = {"Open":"open","High":"high","Low":"low","Close":"close","Adj Close":"adj_close","Volume":"volume"}
#better way that will work regardless of column names and not require janitor
new_col_names = {}
columns = initial_data_pull_df.columns
for col in columns:
    new_col_names.update({col:col.lower().replace(" ","_")})
initial_data_pull_df.rename(columns=new_col_names,inplace=True)

# renaming the index as well to make it lowercase (sadly janitor doesn't yet catch the index name)
initial_data_pull_df.index.rename('date', inplace=True)

# inspecting the tail of our base data
initial_data_pull_df.tail()

,open,high,low,close,adj_close,volume
date,,,,,,
2019-06-17,192.90,194.96,192.17,193.89,193.89,14669100
2019-06-18,196.05,200.29,195.21,198.45,198.45,26551000
2019-06-19,199.68,199.88,197.31,197.87,197.87,21124200
2019-06-20,200.37,200.61,198.03,199.46,199.46,21514000
2019-06-21,198.80,200.85,198.15,198.78,198.78,47735300


## Data Processing

### Trimming to Only Needed Data

In [3]:
# making date a column instead of an index
initial_data_pull_df.reset_index(level=0, inplace=True)

# keeping just date and adjusted closing value, as that's all we need
cleaned_close_df = initial_data_pull_df.loc[:,('date','adj_close')]

In [4]:
# inspecting the base df we'll be working with
cleaned_close_df.tail()

,date,adj_close
365,2019-06-17,193.89
366,2019-06-18,198.45
367,2019-06-19,197.87
368,2019-06-20,199.46
369,2019-06-21,198.78


In [5]:
#going to comment the next one out, trying to replicate in a much cleaner way
cleaned_close_df["recent_ath_val"] = cleaned_close_df.adj_close.cummax()
cleaned_close_df.head()

,date,adj_close,recent_ath_val
0,2018-01-02,168.34,168.34
1,2018-01-03,168.31,168.34
2,2018-01-04,169.09,169.09
3,2018-01-05,171.02,171.02
4,2018-01-08,170.38,171.02


In [6]:
maxDate = cleaned_close_df.groupby("recent_ath_val").date.first().reset_index().rename(columns={"date":"date_of_interest"})
maxDate.head()

close_and_ath_df = cleaned_close_df.merge(maxDate,on="recent_ath_val")
close_and_ath_df.head()

,date,adj_close,recent_ath_val,date_of_interest
0,2018-01-02,168.34,168.34,2018-01-02
1,2018-01-03,168.31,168.34,2018-01-02
2,2018-01-04,169.09,169.09,2018-01-04
3,2018-01-05,171.02,171.02,2018-01-05
4,2018-01-08,170.38,171.02,2018-01-05


### Finding % Down from ATH for Each Value

In [7]:
# need to find an efficient and pythonic way to, for each date, find the date and value of recent ATH
# Zach -- this is the cell of code that definitely needs some optimizing... 
# honestly it can probably be done way better an entirely different way

# instantiate empty df in which to store ATH details
#ath_info_df = pd.DataFrame(columns=['recent_ath_date', 'recent_ath_val', 'date_of_interest'])

# loops over all dates we have, and for each, stores the date and value of the recent all time high
#for date in cleaned_close_df.loc[:,'date']:
    
    # create df of time period to evaluate (all days prior to chosen date)
    #period_to_eval_df = cleaned_close_df.loc[cleaned_close_df.loc[:,'date'] < date, :]

    # check the period to evaluate for ATH and store the date and value in a one-row dataframe
    #recent_ath_row = period_to_eval_df.loc[period_to_eval_df.loc[:,'adj_close'] == period_to_eval_df.loc[:,'adj_close'].max(), :]
    
    # store date of present iteration in the recent_ath_row one-row dataframe
    #recent_ath_row['date_of_interest'] = date
    
    # rename columns so as to not be confusing later on
    #recent_ath_row.rename(index=str, columns={"date": "recent_ath_date", "adj_close": "recent_ath_val"}, inplace=True)
    
    # append the ATH info for present iteration to the df of all ATH infos
    #ath_info_df = ath_info_df.append(recent_ath_row)

In [8]:
# inspecting ATH data that we'll next join back on our base closing data
#ath_info_df.tail()

In [9]:
# merge the main closing data df and the df with ATH info
#close_and_ath_df = pd.merge(
    #left     = cleaned_close_df, 
    #right    = ath_info_df,  
    #left_on  = ['date'], 
    #right_on = ['date_of_interest']
#).drop(
    # drop the date_of_interest column as it is duplicative wrt 'Date' column
    #columns = ['date_of_interest']
#)

In [10]:
# inspecting merged closing and ath data
close_and_ath_df.tail()

,date,adj_close,recent_ath_val,date_of_interest
365,2019-06-17,193.89,229.39,2018-10-03
366,2019-06-18,198.45,229.39,2018-10-03
367,2019-06-19,197.87,229.39,2018-10-03
368,2019-06-20,199.46,229.39,2018-10-03
369,2019-06-21,198.78,229.39,2018-10-03


In [11]:
# calculating the % down from ATH now that we have the raw values needed
close_and_ath_df['perc_down_from_ath'] = (
    100 * (1 - (close_and_ath_df['adj_close'] / close_and_ath_df['recent_ath_val']))
)
close_and_ath_df["time_since_ath"] = close_and_ath_df.date - close_and_ath_df.date_of_interest

In [12]:
close_and_ath_df.tail()

,date,adj_close,recent_ath_val,date_of_interest,perc_down_from_ath,time_since_ath
365,2019-06-17,193.89,229.39,2018-10-03,15.475827,257 days
366,2019-06-18,198.45,229.39,2018-10-03,13.487946,258 days
367,2019-06-19,197.87,229.39,2018-10-03,13.740791,259 days
368,2019-06-20,199.46,229.39,2018-10-03,13.047648,260 days
369,2019-06-21,198.78,229.39,2018-10-03,13.344086,261 days
